In [1]:
from pyspark.sql.functions import col, to_timestamp, current_timestamp
from pyspark.sql.types import DecimalType


StatementMeta(, 13d7e3b1-0a19-41f7-9971-e0f80bb81d32, 3, Finished, Available, Finished)

In [2]:
bronze_df = spark.read.table("bronze_sales")


StatementMeta(, 13d7e3b1-0a19-41f7-9971-e0f80bb81d32, 4, Finished, Available, Finished)

In [5]:
silver_df = (
    bronze_df
    .withColumn("Quantity", col("Quantity").cast(DecimalType(10,2)))
    .withColumn("UnitPrice", col("UnitPrice").cast(DecimalType(10,2)))
    .withColumn("TotalAmount", col("Quantity") * col("UnitPrice"))
    .withColumn("SaleDate", to_timestamp(col("SaleDate")))
    .withColumn("UpdatedDate", to_timestamp(col("UpdatedDate")))
    .withColumn("ingestion_timestamp", current_timestamp())
)


StatementMeta(, 13d7e3b1-0a19-41f7-9971-e0f80bb81d32, 7, Finished, Available, Finished)

In [6]:
(
    silver_df.write
    .format("delta")
    .mode("overwrite")
    .saveAsTable("silver_sales")
)


StatementMeta(, 13d7e3b1-0a19-41f7-9971-e0f80bb81d32, 8, Finished, Available, Finished)

In [7]:
spark.read.table("silver_sales").printSchema()
spark.read.table("silver_sales").show()


StatementMeta(, 13d7e3b1-0a19-41f7-9971-e0f80bb81d32, 9, Finished, Available, Finished)

root
 |-- SaleID: integer (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Product: string (nullable = true)
 |-- Quantity: decimal(10,2) (nullable = true)
 |-- UnitPrice: decimal(10,2) (nullable = true)
 |-- SaleDate: timestamp (nullable = true)
 |-- UpdatedDate: timestamp (nullable = true)
 |-- TotalAmount: decimal(21,4) (nullable = true)
 |-- ingestion_timestamp: timestamp (nullable = true)

+------+----------+--------+--------+---------+-------------------+-------------------+-----------+--------------------+
|SaleID|CustomerID| Product|Quantity|UnitPrice|           SaleDate|        UpdatedDate|TotalAmount| ingestion_timestamp|
+------+----------+--------+--------+---------+-------------------+-------------------+-----------+--------------------+
|     1|       101|  Laptop|    1.00|  1200.00|2026-12-01 00:00:00|2026-12-01 00:00:00|  1200.0000|2026-01-21 11:33:...|
|     2|       102|   Mouse|    2.00|    25.00|2026-12-02 00:00:00|2026-12-02 00:00:00|    50.0000|20